## 챗봇용 데이터
<hr>

1. **식당 정보 데이터**
2. **대화 규칙 데이터**

### 1. 식당 정보 데이터
<hr>

**[사용하는 파일 명]**

1. cat_concat.csv
2. review_concat_rating_revised.csv
<br>
=> 위 2개의 데이터프레임을 이용하여 챗봇에서 사용할 데이터프레임을 만듭니다

In [54]:
### 데이터 불러오기
import pandas as pd

### cat_concat.csv 
df_cat = pd.read_csv('./data/cat_concat.csv')
### review_concat_rating_revised.csv
df_review = pd.read_csv('./data/review_concat_rating_revised.csv')

### 1) cat_concat.csv 전처리

In [55]:
### cat_concat.csv 확인
df_cat.head(1)
# - "restaurant_name", "naver_cat_review", "naver_cat_count" 컬럼만 사용할 예정..

,Unnamed: 0,restaurant_name,ansim,naver_cat_review,naver_cat_count,cat_people
0,0,전포동 베이비치크,없음,음식이 맛있어요,292,314


In [56]:
### 사용할 컬럼만 분리
df_cat = df_cat[["restaurant_name", "naver_cat_review", "naver_cat_count"]]
df_cat.head(1)

,restaurant_name,naver_cat_review,naver_cat_count
0,전포동 베이비치크,음식이 맛있어요,292


In [57]:
### naver_cat_count 정규화
# - 식당별 평가 수가 다르기 떄문에 식당의 경향을 평가하려면 cat_caunt 컬럼을 정규화 하는 것이 좋을 것이라 예상

## 반환값이 들어갈 데이터프레임 만들기
df_cat_scaled = pd.DataFrame()
## 식당명 고유값 리스트에 담기
rest_list = df_cat["restaurant_name"].unique()
## 식당별 카테고리 정규화 진행해서 df_cat_scaled에 담기
for rest in rest_list:
    temp = df_cat[(df_cat["restaurant_name"]==rest)==True] # 식당별 데이터프레임 필터링
    cat_sum = temp["naver_cat_count"].sum()                # 식당별 naver_cat_count의 합계 구하기
    temp["normalized"] = temp[(temp["restaurant_name"]==rest)==True]["naver_cat_count"] / cat_sum # 정규화 컬럼 추가
    df_cat_scaled = pd.concat([df_cat_scaled, temp], axis=0) # 새로운 데이터프레임에 담기

C:\Users\USER\AppData\Local\Temp\ipykernel_18776\2090295809.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["normalized"] = temp[(temp["restaurant_name"]==rest)==True]["naver_cat_count"] / cat_sum # 정규화 컬럼 추가
C:\Users\USER\AppData\Local\Temp\ipykernel_18776\2090295809.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["normalized"] = temp[(temp["restaurant_name"]==rest)==True]["naver_cat_count"] / cat_sum # 정규화 컬럼 추가
C:\Users\USER\AppData\Local\Temp\ipykernel_18776\2090295809.py:12: Setting

In [67]:
df_cat_scaled

,restaurant_name,naver_cat_review,naver_cat_count,normalized
0,전포동 베이비치크,음식이 맛있어요,292,0.225483
1,전포동 베이비치크,인테리어가 멋져요,230,0.177606
2,전포동 베이비치크,친절해요,179,0.138224
3,전포동 베이비치크,특별한 날 가기 좋아요,121,0.093436
4,전포동 베이비치크,특별한 메뉴가 있어요,99,0.076448
...,...,...,...,...
18445,가야동 코코샤브가야점,특별한 날 가기 좋아요,4,0.052632
18446,가야동 코코샤브가야점,양이 많아요,3,0.039474
18447,가야동 코코샤브가야점,가성비가 좋아요,2,0.026316
18448,가야동 코코샤브가야점,단체모임 하기 좋아요,2,0.026316


In [59]:
### 업소명에 '호텔' 들어간거 빼기
df_cat_scaled = df_cat_scaled[~df_cat_scaled['restaurant_name'].str.contains('호텔')]

#### cat_concat.csv 전처리 완료

In [60]:
df_cat_scaled.to_csv("./preprocessed/category_count.csv",index=False)

### 1-1) 식당별 총 평가 수 데이터프레임 추출

In [133]:
cnt_sum = df_cat_scaled[(df_cat_scaled["restaurant_name"]=="가야동 코코샤브가야점")==True]["naver_cat_count"].sum()

76

In [139]:
### 
df_cat_sum = pd.DataFrame()
for rest in rest_list:
    cnt_sum = df_cat_scaled[(df_cat_scaled["restaurant_name"]==rest)==True]["naver_cat_count"].sum()
    temp = pd.DataFrame([[rest, cnt_sum]], columns=["restaurant_name", "naver_cat_count_sum"])
    df_cat_sum = pd.concat([df_cat_sum, temp], axis=0)
df_cat_sum.sort_values("naver_cat_count_sum", ascending=False).reset_index(drop=True).to_csv("./preprocessed/cat_cum.csv", index=False)

In [136]:
df_cat_sum.describe()

,naver_cat_count_sum
count,1535.000000
mean,448.862541
std,1224.970349
min,0.000000
25%,84.000000
50%,193.000000
75%,446.000000
max,34921.000000


### 2) review_concat_rating_revised.csv 전처리

In [50]:
df_review

,Unnamed: 0,restaurant_name,review_content,rating,date,visit,register,category
0,0,전포동 베이비치크,아기자기하고 빈티지한 분위기가 좋았어요ㅎㅎ 명란청양오일파스타랑 트러플버섯크림리조또 ...,NaN,2023-08-02,1,영수증,양식
1,1,전포동 베이비치크,입구부터 이쁜 공간이 너무 많아서 좋았던 이곳 취향저격 제대로 당해서 안으로 들어왔...,NaN,2023-08-01,1,영수증,양식
2,2,전포동 베이비치크,와..오랜만에 너무 만족스러운 식사하고 왔어여ㅠ 매장 분위기도 너무 좋고 예쁘신 사...,NaN,2023-07-24,1,영수증,양식
3,3,전포동 베이비치크,지난번에 너무 맛있게 먹고 이번에 재방문했어요! 베이비치크 플레이트는 진짜 무조건 ...,NaN,2023-08-09,1,영수증,양식
4,4,전포동 베이비치크,양 진짜 많고 너무 맛있어요ㅠㅠ 안에도 아기자기 앤티크하게 잘 꾸며져있고ㅠㅠㅠ 감성...,NaN,2023-07-05,1,영수증,양식
...,...,...,...,...,...,...,...,...
116259,116259,양정동 명심이족발,맛있어요,5.0,2020-12-19,1,영수증,"족발,보쌈"
116260,116260,양정동 명심이족발,맛있고 양도 많았어요!,5.0,2020-12-25,2,영수증,"족발,보쌈"
116261,116261,양정동 명심이족발,좋아요,3.5,2020-10-29,1,영수증,"족발,보쌈"
116262,116262,양정동 명심이족발,맛있어요,3.5,2019-12-15,1,영수증,"족발,보쌈"


#### 위의 데이터를 활용하여 식당별 업태 구분명 추출

In [51]:
### 식당과 업태 구분만 데이터프레임에 담기
df_uptae = df_review[["restaurant_name", "category"]].drop_duplicates().reset_index(drop=True)
df_uptae

,restaurant_name,category
0,전포동 베이비치크,양식
1,전포동 제이에스푸드,한식
2,양정동 론도테이블,"카페,디저트"
3,전포동 하업트(HAUPT),바(BAR)
4,부전동 야타이 수,이자카야
...,...,...
1806,연지동 선명아구찜,한식
1807,부전동 중천금,중식당
1808,가야동 황금어장횟집,생선회
1809,부전동 뱃살도둑,"다이어트,샐러드"


In [83]:
### 동(dong) 컬럼 추가
df_uptae["dong"]=""

for idx in range(len(df_uptae["restaurant_name"])):
    dong = df_uptae.loc[idx, "restaurant_name"].split(" ")[0]
    df_uptae.loc[idx, "dong"] = dong
df_uptae

,restaurant_name,category,dong
0,전포동 베이비치크,양식,전포동
1,전포동 제이에스푸드,한식,전포동
2,양정동 론도테이블,"카페,디저트",양정동
3,전포동 하업트(HAUPT),바(BAR),전포동
4,부전동 야타이 수,이자카야,부전동
...,...,...,...
1806,연지동 선명아구찜,한식,연지동
1807,부전동 중천금,중식당,부전동
1808,가야동 황금어장횟집,생선회,가야동
1809,부전동 뱃살도둑,"다이어트,샐러드",부전동


In [213]:
df_uptae["category"].unique()

array(['양식', '한식', '카페,디저트', '바(BAR)', '이자카야', '한식뷔페', '다이어트,샐러드',
       '브런치카페', '일식당', '포장마차', '요리주점', '술집', '케이크전문', '소고기구이', '돈가스',
       '치킨,닭강정', '국수', '백숙,삼계탕', '육류,고기요리', '복어요리', '샌드위치', '초밥,롤', '김밥',
       '주꾸미요리', '카페', '낙지요리', '족발,보쌈', '브런치', '종합분식', '스파게티,파스타전문', '와인',
       '생선회', '중식당', '샤브샤브', '햄버거', '오리요리', '전,빈대떡', '죽', '보드카페',
       '전통,민속주점', '식료품', '덮밥', '피자', '돼지고기구이', '토스트', '맥주,호프', '칼국수,만두',
       '아귀찜,해물찜', '인테리어소품', '일본식라면', '아시아음식', '태국음식', '비빔밥', '해물,생선요리',
       '베트남음식', '주류', '닭요리', '백반,가정식', '이벤트,파티', '곱창,막창,양', '보리밥', '국밥',
       '만두', '한정식', '곰탕,설렁탕', '두부요리', '감자탕', '양꼬치', '추어탕', '매운탕,해물탕',
       '마라탕', '멕시코,남미음식', '도시락,컵밥', '이불,담요', '애견카페', '찌개,전골', '일식튀김,꼬치',
       '냉면', '쌈밥', '조개요리', '퓨전음식', '종합도소매', '생선구이', '분식', '장어,먹장어요리',
       '딤섬,중식만두', '테이크아웃커피', '호두과자', '닭발', '아이스크림', '스테이크,립', '룸카페', '급식',
       '찜닭', '이탈리아음식', '과일,주스전문점', '과자,사탕,초코렛', '닭볶음탕', '귀금속,시계', 'DDC치킨',
       '대게요리', '반찬가게', '도넛', '사철,영양탕', '카레', '해장국', '갈비탕', '기사식당', '

In [121]:
comma_list = []
for cat in df_uptae["category"].unique():
    if "," in cat:
        comma_list.append(cat)
print(comma_list)

['카페,디저트', '다이어트,샐러드', '치킨,닭강정', '백숙,삼계탕', '육류,고기요리', '초밥,롤', '족발,보쌈', '스파게티,파스타전문', '전,빈대떡', '전통,민속주점', '맥주,호프', '칼국수,만두', '아귀찜,해물찜', '해물,생선요리', '백반,가정식', '이벤트,파티', '곱창,막창,양', '곰탕,설렁탕', '매운탕,해물탕', '멕시코,남미음식', '도시락,컵밥', '이불,담요', '찌개,전골', '일식튀김,꼬치', '장어,먹장어요리', '딤섬,중식만두', '스테이크,립', '과일,주스전문점', '과자,사탕,초코렛', '귀금속,시계', '사철,영양탕', '오뎅,꼬치', '순대,순댓국', '네일아트,네일샵', '수산물가공,제조', '우동,소바']


In [128]:
df_1234 = df_uptae["category"].unique()

In [127]:
df_1234 = pd.DataFrame(df_uptae["category"].unique(), columns=["request"])
df_1234.to_excel("./preprocessed/늦은_시간에_고생이_많으십니다.xlsx", index=False)

In [53]:
df_uptae.to_csv("./preprocessed/category_dong.csv", index=False)

<br>
<br>
<br>



### 2.대화 규칙 데이터

In [68]:
### 대화 데이터 불러오기
chatbot_data = pd.read_excel("./data/chatbot_data_sample.xlsx")
chatbot_data

,request,rule,response
0,너의 이름은?,너|이름,저는 자비스라고 합니다
1,네 이름을 말해줘,네|이름|말해,저는 자비스라고 합니다
2,네 이름이 뭐니?,네|이름|뭐,저는 자비스라고 합니다
3,놀러가고 싶다,놀러|싶,가끔씩 휴식하는 것도 좋죠
4,느그 아부지 뭐하시노,느그|아부지|뭐하,우리 아부지 건달입니다
5,말귀좀 알아듣는다?,말귀|알아듣는다,다행이네요. 열심히 배우고 있어요
6,맛저해,맛저해,맛저하세요~
7,맛점해,맛점해,맛점하세요~
8,메리크리스마스,메리|크리,메리~ 크리스마스~
9,면접에서 떨어졌어,면접|떨어,다음엔 꼭 붙을 수 있을거에요


#### 2-2) 동별 데이터

In [116]:
### 부산 진구 내 동별 데이터 추가하기

## 1. df_uptae["dong"]에서 동만 빼고 리스트에 담기
list_dong = []
for dong in df_uptae["dong"].unique():
    list_dong.append(dong[:-1])
#list_dong

### 2. 위 데이터에 추가하기
chatbot_dong = pd.DataFrame()

for idx in range(len(list_dong)):
    temp = pd.DataFrame()
    #print(list_dong[idx])
    temp["request"] = [list_dong[idx]+"동"]
    temp["rule"] = [list_dong[idx]]
    temp["response"] = [list_dong[idx]+"동"]
    chatbot_dong = pd.concat([chatbot_dong, temp], axis=0)
chatbot_dong = chatbot_dong.reset_index(drop=True)
chatbot_dong

,request,rule,response
0,전포동,전포,전포동
1,양정동,양정,양정동
2,부전동,부전,부전동
3,개금동,개금,개금동
4,가야동,가야,가야동
5,당감동,당감,당감동
6,범천동,범천,범천동
7,연지동,연지,연지동
8,초읍동,초읍,초읍동
9,부암동,부암,부암동


In [129]:
chatbot_dong.to_csv("./preprocessed/dong.csv", index=False)

#### 2-3)가게명&동 제거한 가게명 데이터

In [178]:
chatbot_rest = pd.DataFrame(data=df_uptae["restaurant_name"] ,columns=["restaurant_name"])

for idx in range(len(df_uptae["restaurant_name"])):
    rest = df_uptae.loc[idx, "restaurant_name"].split(" ")[1] # restaurant_name에서 동 제거
    rest = rest.split("(")[0] # 식당명(영어)형식에서 (영어)제거
    chatbot_rest.loc[idx, "delete_dong_restaurant_name"] = rest # chatbot_rest에 추가

chatbot_rest

,restaurant_name,delete_dong_restaurant_name
0,전포동 베이비치크,베이비치크
1,전포동 제이에스푸드,제이에스푸드
2,양정동 론도테이블,론도테이블
3,전포동 하업트(HAUPT),하업트
4,부전동 야타이 수,야타이
...,...,...
1806,연지동 선명아구찜,선명아구찜
1807,부전동 중천금,중천금
1808,가야동 황금어장횟집,황금어장횟집
1809,부전동 뱃살도둑,뱃살도둑


In [179]:
rest_name_list = df_uptae["restaurant_name"].to_list()
len(rest_name_list)

1811

In [205]:
df = pd.DataFrame(columns=["request", "rule", "response"])
rest_name_list = df_uptae["restaurant_name"].to_list()

for idx in range(len(rest_name_list)):
    df.loc[idx, "request"] = rest_name_list[idx]
    df.loc[idx, "rule"] = "|".join([df_uptae.loc[idx, "restaurant_name"].split(" ")[0], chatbot_rest.loc[idx, "delete_dong_restaurant_name"]])
    df.loc[idx, "response"] = rest_name_list[idx]
    temp = pd.DataFrame(data=[[rest_name_list[idx],
                               chatbot_rest.loc[idx, "delete_dong_restaurant_name"],
                               rest_name_list[idx]]], 
                        columns=["request", "rule", "response"])
    df = pd.concat([df, temp], axis=0)
df

,request,rule,response
0,전포동 베이비치크,전포동|베이비치크,전포동 베이비치크
0,전포동 베이비치크,베이비치크,전포동 베이비치크
1,전포동 제이에스푸드,전포동|제이에스푸드,전포동 제이에스푸드
0,전포동 제이에스푸드,제이에스푸드,전포동 제이에스푸드
2,양정동 론도테이블,양정동|론도테이블,양정동 론도테이블
...,...,...,...
0,가야동 황금어장횟집,황금어장횟집,가야동 황금어장횟집
1809,부전동 뱃살도둑,부전동|뱃살도둑,부전동 뱃살도둑
0,부전동 뱃살도둑,뱃살도둑,부전동 뱃살도둑
1810,양정동 명심이족발,양정동|명심이족발,양정동 명심이족발


In [206]:
df = df.reset_index(drop=True)
df

,request,rule,response
0,전포동 베이비치크,전포동|베이비치크,전포동 베이비치크
1,전포동 베이비치크,베이비치크,전포동 베이비치크
2,전포동 제이에스푸드,전포동|제이에스푸드,전포동 제이에스푸드
3,전포동 제이에스푸드,제이에스푸드,전포동 제이에스푸드
4,양정동 론도테이블,양정동|론도테이블,양정동 론도테이블
...,...,...,...
3617,가야동 황금어장횟집,황금어장횟집,가야동 황금어장횟집
3618,부전동 뱃살도둑,부전동|뱃살도둑,부전동 뱃살도둑
3619,부전동 뱃살도둑,뱃살도둑,부전동 뱃살도둑
3620,양정동 명심이족발,양정동|명심이족발,양정동 명심이족발


In [208]:
### 결측치 상호명 대체
# - [445, 531, 1343, 3559] 행에 결측치 존재
# - [444, 530, 1342, 3558] 행도 수정
rule1 = ["부전동|사미헌", "개금동|용화집", "부전동|오매족", "부전동|우미남"]
rule2 = ["사미헌", "용화집", "오매족", "우미남"]
a=0
for idx in [444, 530, 1342, 3558]:
    df.loc[idx, "rule"] = rule1[a]
    df.loc[idx+1, "rule"] = rule2[a]

    a+=1
df

,request,rule,response
0,전포동 베이비치크,전포동|베이비치크,전포동 베이비치크
1,전포동 베이비치크,베이비치크,전포동 베이비치크
2,전포동 제이에스푸드,전포동|제이에스푸드,전포동 제이에스푸드
3,전포동 제이에스푸드,제이에스푸드,전포동 제이에스푸드
4,양정동 론도테이블,양정동|론도테이블,양정동 론도테이블
...,...,...,...
3617,가야동 황금어장횟집,황금어장횟집,가야동 황금어장횟집
3618,부전동 뱃살도둑,부전동|뱃살도둑,부전동 뱃살도둑
3619,부전동 뱃살도둑,뱃살도둑,부전동 뱃살도둑
3620,양정동 명심이족발,양정동|명심이족발,양정동 명심이족발


In [214]:
df[(df["rule"]=="플라스티카")==True]

,request,rule,response
3409,전포동 플라스티카(PLASTICA),플라스티카,전포동 플라스티카(PLASTICA)


In [212]:
df.reset_index(drop=True).to_csv("./preprocessed/chatbot_rest.csv", index=True, encoding="utf-8")